# FIT5196 Data wrangling 
## Assignment 4 - Text Preprocessing



### Author: Ramprasath Karunakaran

ID: 26994437

Date written: 18/10/2016

Version: 1.0

Program: Python 2.7.12 and Jupyter notebook

In [1]:
#checking python version
!python --version

Python 2.7.12 :: Anaconda 4.1.1 (x86_64)


In [2]:
# importing libraries that will be used in this report
import pandas as pd
import re
import numpy as np
import os
import nltk
import xml.etree.ElementTree as ET
import csv

## Task 1 - Parsing the XML files

The given patent is completely in the XML format. From these XML, we have to extract the following details:
> - Patent ID
- Section ID
- Abstract
- Description
- Claims

In [3]:
#folder which has all the patent files
patents_folder="./100"

In [4]:
#to create a dictionary for each patent using object instantiation method
class Patent(object):
    def __init__(self, documentID, sectionID, abstract, description, claims):
        self.documentID = documentID
        self.sectionID = sectionID
        self.abstract = abstract
        self.description = description
        self.claims = claims

**All the functions used below are derived from the three chapters of Fundamentals of Text Preprocessing given in Alexandria materials.**

The given XML files can be parsed using the Element Tree in Python's XML library. Hence, all the details will be stored in a tree structure where each XML tag serves as a node.

In [6]:
#empty list to store all the patents
patentList =[]
#iterating through all the files and subfolders in the folder
for root,subfolders,files in os.walk(patents_folder):
    for file_name in files:
        #combining all the file path which will be fed to element tree for parsing
        file_path  = os.path.join(root, file_name)
        # to parse only xml files
        if file_path.endswith('XML'):
            # converting the file to a xml tree
            tree = ET.parse(file_path)
            # locating the required nodes and storing them in variables
            documentID = tree.find('.//doc-number').text
            sectionID = tree.find('.//section').text
            abst = tree.find('.//abstract')
            abstract = ET.tostring(abst, method="text",encoding='UTF-8') #XML uses UTF-8 codec
            desc = tree.find('.//description')
            description = ET.tostring(desc, method="text",encoding='UTF-8')
            clm = tree.find('.//claims')
            claims = ET.tostring(clm, method="text",encoding='UTF-8')
            # create a patent dictionary by creating a new object
            patent = Patent(documentID, sectionID, abstract, description,claims)
            # appending the patent to the list
            patentList.append(patent)

Once all the XML files are parsed using trees. All the patentIDs and the section IDs are stored in a text file called 'section_labels.txt'

In [7]:
# extracting the document ID and section ID alone from the patentList
labelInfo = dict([(patent.documentID, patent.sectionID) for patent in patentList])
# opening a new csv file in the write mode
labelFile = csv.writer(open("section_labels.txt","w"))
for key, value in labelInfo.items():
    # reading every key value and writing them into a file
    labelFile.writerow([key, value])

## Task 2 - Tokenisation of Text

From the patent list created above, the abstract, description and claims sections are chosen to be tokenised. Hence we create a dictionary with patent IDs and their respective text details.

In [8]:
#extracting text of patents with document IDs
patentText = dict([ (patent.documentID ,",".join((patent.abstract,patent.description,patent.claims))) for patent in patentList])

All the given files are patent files which are properly formatted using XML. Hence, there were no special characters involved in those documents. If there were some special characters, we will be in need of regular expressions to customise the process of tokenisation. So, we use the tokeniser library available in Natural Language ToolKit(NLTK) which will tokenise the entire dictionary 'patentText' choosing only the alphabet letters in the text.

In [11]:
def tokenize_sent(sent):
    """
    The function tokenizes a sentence, and return a list of words that only contain alphabet 
    letters.
    """
    #lower() converts the words to lowercase which will avoid confusion while creating a dictionary of vocabulary
    return [word for word in nltk.word_tokenize(sent.lower()) if word.isalpha()]

In [12]:
# dictionary to store patent ids and their corresponding set of tokens
tokenized_sents = {} 
# iterating through the patent text
for keys in patentText.iterkeys():
    #decode function is to support the ascii codec in python and all tokens are in UTF-8 encoding 
    tokenized_sents[keys] = tokenize_sent(patentText[keys].decode('utf-8')) 

Hence, all the patent IDs and their respective tokens are stored in a dictionary for further processing.

Now, we try to find the most common tokens in the data using the FreqDist function in NLTK probability.

In [13]:
from nltk.probability import *

def word_concat(dsd):
    """
    concatenate all the words stored in the values of a given dictionary. Each value is a list
    of tokenized sentences.
    """
    all_words = []
    # iterates all patents and store the tokens in a single list.
    for sent in dsd.values():
            all_words += sent
    print "tokens:", len(all_words) # total number of tokens 
    print "types:", len(set(all_words)) # total number of types
    return all_words

In [14]:
#listing the common words and their number of occurences
freq_dist = FreqDist(word_concat(tokenized_sents))
freq_dist.most_common(20)

tokens: 5090460
types: 34458


[(u'the', 487784),
 (u'of', 223846),
 (u'a', 199162),
 (u'to', 135462),
 (u'and', 132328),
 (u'in', 112870),
 (u'is', 95906),
 (u'be', 49294),
 (u'for', 45054),
 (u'an', 42127),
 (u'or', 41939),
 (u'as', 41653),
 (u'with', 37510),
 (u'that', 34289),
 (u'by', 33473),
 (u'are', 29809),
 (u'at', 29290),
 (u'from', 28798),
 (u'may', 26975),
 (u'fig', 25426)]

In [22]:
bigram=[]
trigram=[]
for keys in tokenized_sents.iterkeys():
    bigram=nltk.bigrams(tokenized_sents[keys])

In [23]:
list(bigram)

[(u'a', u'pixel'),
 (u'pixel', u'with'),
 (u'with', u'a'),
 (u'a', u'photosensor'),
 (u'photosensor', u'and'),
 (u'and', u'a'),
 (u'a', u'transfer'),
 (u'transfer', u'transistor'),
 (u'transistor', u'having'),
 (u'having', u'a'),
 (u'a', u'split'),
 (u'split', u'transfer'),
 (u'transfer', u'gate'),
 (u'gate', u'a'),
 (u'a', u'first'),
 (u'first', u'section'),
 (u'section', u'of'),
 (u'of', u'the'),
 (u'the', u'transfer'),
 (u'transfer', u'gate'),
 (u'gate', u'is'),
 (u'is', u'connectable'),
 (u'connectable', u'to'),
 (u'to', u'a'),
 (u'a', u'first'),
 (u'first', u'voltage'),
 (u'voltage', u'source'),
 (u'source', u'while'),
 (u'while', u'a'),
 (u'a', u'second'),
 (u'second', u'section'),
 (u'section', u'of'),
 (u'of', u'the'),
 (u'the', u'transfer'),
 (u'transfer', u'gate'),
 (u'gate', u'is'),
 (u'is', u'connectable'),
 (u'connectable', u'to'),
 (u'to', u'a'),
 (u'a', u'second'),
 (u'second', u'voltage'),
 (u'voltage', u'source'),
 (u'source', u'thus'),
 (u'thus', u'during'),
 (u'durin

From the above frequencies, it is evident that stop words like 'the','of' are going to affect our process of preprocessing since they do not provide much value to the processing.

So we try to remove the stopwords before finding bigrams and trigrams.

In [24]:
# import set of 127 stop words from NLTK corpus
from nltk.corpus import stopwords
stopwords_list = stopwords.words('english')
stopwords_list

[u'i',
 u'me',
 u'my',
 u'myself',
 u'we',
 u'our',
 u'ours',
 u'ourselves',
 u'you',
 u'your',
 u'yours',
 u'yourself',
 u'yourselves',
 u'he',
 u'him',
 u'his',
 u'himself',
 u'she',
 u'her',
 u'hers',
 u'herself',
 u'it',
 u'its',
 u'itself',
 u'they',
 u'them',
 u'their',
 u'theirs',
 u'themselves',
 u'what',
 u'which',
 u'who',
 u'whom',
 u'this',
 u'that',
 u'these',
 u'those',
 u'am',
 u'is',
 u'are',
 u'was',
 u'were',
 u'be',
 u'been',
 u'being',
 u'have',
 u'has',
 u'had',
 u'having',
 u'do',
 u'does',
 u'did',
 u'doing',
 u'a',
 u'an',
 u'the',
 u'and',
 u'but',
 u'if',
 u'or',
 u'because',
 u'as',
 u'until',
 u'while',
 u'of',
 u'at',
 u'by',
 u'for',
 u'with',
 u'about',
 u'against',
 u'between',
 u'into',
 u'through',
 u'during',
 u'before',
 u'after',
 u'above',
 u'below',
 u'to',
 u'from',
 u'up',
 u'down',
 u'in',
 u'out',
 u'on',
 u'off',
 u'over',
 u'under',
 u'again',
 u'further',
 u'then',
 u'once',
 u'here',
 u'there',
 u'when',
 u'where',
 u'why',
 u'how',
 u'all

Remove the above stop words from the tokenised sentences.

In [25]:
for keys in tokenized_sents.iterkeys():
    filtered_sents = [token for token in tokenized_sents[keys] if token not in stopwords_list ]

Furthermore, we import another set of Stopwords from [Kevin Bouge's website](https://sites.google.com/site/kevinbouge/stopwords-lists) and store it in a separate text file and remove them from the data. 

In [26]:
import nltk
#list to store the stop words
stopwords_list_570 = []
# all the stopwords are stored in a text file
with open('./stopwords_en.txt') as f:
    stopwords_list_570 = f.read().splitlines()

In [32]:
# removing the 570 stop words
filtered_sents = [w for w in tokenized_sents[keys] if w.lower() not in stopwords_list_570]
removed_sents=[w for w in tokenized_sents[keys] if w.lower() in stopwords_list_570]
diff = [w for w in removed_sents if w.lower() not in stopwords_list]

In [33]:
diff

[u'first',
 u'first',
 u'second',
 u'second',
 u'thus',
 u'two',
 u'may',
 u'within',
 u'two',
 u'may',
 u'herein',
 u'particularly',
 u'use',
 u'used',
 u'one',
 u'associated',
 u'within',
 u'may',
 u'may',
 u'also',
 u'necessary',
 u'known',
 u'known',
 u'example',
 u'et',
 u'et',
 u'see',
 u'also',
 u'herein',
 u'four',
 u'also',
 u'taken',
 u'along',
 u'may',
 u'within',
 u'one',
 u'associated',
 u'may',
 u'many',
 u'different',
 u'along',
 u'one',
 u'example',
 u'causes',
 u'may',
 u'overall',
 u'causes',
 u'associated',
 u'also',
 u'already',
 u'causes',
 u'near',
 u'one',
 u'possible',
 u'accordingly',
 u'likely',
 u'however',
 u'tends',
 u'another',
 u'still',
 u'even',
 u'though',
 u'several',
 u'may',
 u'across',
 u'may',
 u'actually',
 u'likely',
 u'however',
 u'thus',
 u'moreover',
 u'therefore',
 u'without',
 u'also',
 u'brief',
 u'described',
 u'various',
 u'provides',
 u'first',
 u'first',
 u'second',
 u'second',
 u'thus',
 u'two',
 u'may',
 u'within',
 u'two',
 u'may',


In [29]:
from nltk.probability import *
freq_dist1 = FreqDist(removed_sents)
freq_dist1.most_common(20)

[(u'the', 414),
 (u'a', 248),
 (u'of', 218),
 (u'to', 140),
 (u'and', 111),
 (u'is', 98),
 (u'in', 87),
 (u'for', 50),
 (u'an', 49),
 (u'be', 45),
 (u'second', 42),
 (u'as', 37),
 (u'first', 34),
 (u'with', 34),
 (u'may', 34),
 (u'are', 31),
 (u'by', 28),
 (u'which', 27),
 (u'on', 23),
 (u'from', 22)]

Now, we try to list the most common words from the tokens using the same function as above. 

In [34]:
from nltk.probability import *
freq_dist = FreqDist(filtered_sents)
freq_dist.most_common(20)

[(u'pixel', 83),
 (u'region', 77),
 (u'transfer', 67),
 (u'gate', 65),
 (u'transistor', 65),
 (u'cell', 55),
 (u'fig', 51),
 (u'electrode', 42),
 (u'exemplary', 41),
 (u'charge', 36),
 (u'voltage', 34),
 (u'invention', 32),
 (u'section', 29),
 (u'photodiode', 29),
 (u'forming', 26),
 (u'diffusion', 23),
 (u'sections', 23),
 (u'substrate', 22),
 (u'floating', 22),
 (u'source', 20)]

Now, we try to find bigrams and trigrams using the NLTK in-built methods.

In [35]:
#finding bigrams and trigrams
my_bigrams = nltk.bigrams(filtered_sents)
my_trigrams = nltk.trigrams(filtered_sents)

In [36]:
#storing bigrams and trigrams in a list
list_bigrams=list(my_bigrams)
list_trigrams=list(my_trigrams)

In [37]:
# trying to find the frequency of bigrams and trigrams
common_trigrams = FreqDist(list_trigrams)
common_bigrams = FreqDist(list_bigrams)

Now, we extract the top 200 bigrams and top 100 trigrams to retokenise.

In [38]:
# extract the top 200 bigram and 100 trigram based on the frequency distribution
trigrams =[item[0] for item in common_trigrams.most_common(100)]
bigrams = [item[0] for item in common_bigrams.most_common(200)]

In [40]:
bigrams

[(u'pixel', u'cell'),
 (u'transfer', u'transistor'),
 (u'diffusion', u'region'),
 (u'floating', u'diffusion'),
 (u'electrode', u'section'),
 (u'transfer', u'gate'),
 (u'gate', u'electrode'),
 (u'transistor', u'gate'),
 (u'exemplary', u'embodiment'),
 (u'exemplary', u'pixel'),
 (u'fig', u'view'),
 (u'electrode', u'sections'),
 (u'isolation', u'region'),
 (u'dark', u'current'),
 (u'accumulation', u'region'),
 (u'method', u'claim'),
 (u'constructed', u'accordance'),
 (u'integration', u'period'),
 (u'cell', u'fig'),
 (u'embodiment', u'invention'),
 (u'accordance', u'exemplary'),
 (u'follower', u'transistor'),
 (u'source', u'follower'),
 (u'region', u'substrate'),
 (u'gate', u'oxide'),
 (u'reset', u'transistor'),
 (u'voltage', u'source'),
 (u'shown', u'fig'),
 (u'invention', u'fig'),
 (u'charge', u'transfer'),
 (u'section', u'transfer'),
 (u'sensing', u'node'),
 (u'cell', u'constructed'),
 (u'cmos', u'image'),
 (u'storage', u'region'),
 (u'connectable', u'voltage'),
 (u'surface', u'substrat

Now, we retokenise the given text by adding the bigrams and trigrams to our set of tokens using Multiword Tokeniser in NLTK.

In [42]:
# Merge the list of trigram and bigram
multiwordlist = []
multiwordlist.extend(bigrams)
multiwordlist.extend(trigrams)

In [43]:
from nltk.tokenize import MWETokenizer
mwe_tokens = {}
for keys in tokenized_sents.iterkeys():
    mwe_tokenizer = MWETokenizer(multiwordlist)
    # applying the trigram and bigram to the patent dict with stop words
    mwe_tokens[keys] = mwe_tokenizer.tokenize(tokenized_sents[keys])

In [44]:
mwe_tokens['07640605']

[u'that',
 u'end',
 u'of',
 u'a',
 u'support',
 u'for',
 u'a',
 u'shower',
 u'head',
 u'that',
 u'is',
 u'coordinated',
 u'to',
 u'the',
 u'wall',
 u'may',
 u'be',
 u'pivoted',
 u'about',
 u'a',
 u'horizontal',
 u'axis',
 u'a',
 u'holder',
 u'into',
 u'which',
 u'a',
 u'shower',
 u'head',
 u'may',
 u'be',
 u'inserted',
 u'is',
 u'provided',
 u'on',
 u'the',
 u'far',
 u'end',
 u'of',
 u'the',
 u'arm',
 u'that',
 u'holder',
 u'may',
 u'also',
 u'be',
 u'pivoted',
 u'about',
 u'a',
 u'horizontal',
 u'axis',
 u'background',
 u'of',
 u'the',
 u'invention',
 u'the',
 u'invention',
 u'is',
 u'based',
 u'on',
 u'an',
 u'arrangement',
 u'by',
 u'which',
 u'a',
 u'shower',
 u'head',
 u'which',
 u'may',
 u'be',
 u'detachable',
 u'if',
 u'desired',
 u'is',
 u'supported',
 u'such',
 u'that',
 u'it',
 u'may',
 u'be',
 u'pivoted',
 u'that',
 u'shower',
 u'head',
 u'may',
 u'be',
 u'attached',
 u'to',
 u'a',
 u'console',
 u'using',
 u'for',
 u'example',
 u'conical',
 u'holders',
 u'whose',
 u'heights'

In [58]:
not_english

[u'coordinated',
 u'pivoted',
 u'pivoted',
 u'supported',
 u'pivoted',
 u'using',
 u'holders',
 u'heights',
 u'allows',
 u'adjusting',
 u'heights',
 u'having',
 u'heights',
 u'pivoting',
 u'pivotable',
 u'pivotable',
 u'takes',
 u'configured',
 u'profiled',
 u'cf',
 u'inserting',
 u'arranged',
 u'has',
 u'creating',
 u'constructed',
 u'provides',
 u'users',
 u'showering',
 u'opportunities',
 u'proposes',
 u'having',
 u'features',
 u'claims',
 u'elaborations',
 u'subclaims',
 u'elaborating',
 u'pivoted',
 u'pivoted',
 u'pivots',
 u'allows',
 u'orienting',
 u'exiting',
 u'users',
 u'desires',
 u'proves',
 u'elaborating',
 u'pivots',
 u'presumed',
 u'pivots',
 u'orientations',
 u'fastened',
 u'elaborating',
 u'passes',
 u'exiting',
 u'pivoted',
 u'pivoted',
 u'elaborating',
 u'configured',
 u'arranged',
 u'tines',
 u'forking',
 u'elaborating',
 u'intersects',
 u'embodiments',
 u'remaining',
 u'exiting',
 u'changed',
 u'pivoting',
 u'elaborating',
 u'configured',
 u'acts',
 u'means',
 u'at

Now, we have a dictionary of patent IDs and their respective tokens(multiwords implemented).

## Task 3 - Vectors
To implement vectors, we use the tokens without removing stop words and we generate a list of sentences for each patent by combining all the tokens. These sentences will make it easy for us to fit the data in vectoriser functions.

In [47]:
#list of patent ids
sents_ids = []
#list of sentences
sents_words =[]
for key, value in mwe_tokens.items():
    sents_ids.append("{0}".format(key))
    txt = ' '.join(value) # joining all the tokens to form a sentence
    sents_words.append(txt) # appending the sentences to the list

## TF-IDF Vectors

To produce TF-IDF vectors, we use TfidfVectorizer from Scikit library which will fit the sentences we formed above to get a weight factor matrix.

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer is set to analyse word by word
tfidf_vectorizer = TfidfVectorizer(input = 'content', analyzer = 'word')
#fitting the sentences into the vectorizer
tfidf_vectors = tfidf_vectorizer.fit_transform(sents_words)
tfidf_vectors.shape

(800, 34415)

In [52]:
tfidf_vectorizer.get_feature_names()

[u'aa',
 u'aaa',
 u'aaaaa',
 u'aaaaaaaaaaaaaaaaaaaa',
 u'aaabcccc',
 u'aaactgcagcccgggtccagctggcaggcaccctg',
 u'aaagaggtctacatgtcc',
 u'aabbxx',
 u'aac',
 u'aacgaattatggtcacat',
 u'aacgggataactgcacct',
 u'aacgttggctaccctgcacg',
 u'aactgtcttatcttcttttgta',
 u'aagaacatgtgtaagctgcggccc',
 u'aaggagccgattttacta',
 u'aan',
 u'aapf',
 u'aapi',
 u'aatgcatcaacatcttcagg',
 u'aav',
 u'ab',
 u'aba',
 u'abaca',
 u'abandoned',
 u'abaqus',
 u'abate',
 u'abated',
 u'abatement',
 u'abates',
 u'abating',
 u'abb',
 u'abbott',
 u'abbreviated',
 u'abbreviation',
 u'abbreviations',
 u'abc',
 u'abcd',
 u'abcdefg',
 u'abd',
 u'abdominal',
 u'abduct',
 u'abducted',
 u'abducting',
 u'abduction',
 u'abe',
 u'abellows',
 u'aberrant',
 u'aberrated',
 u'aberration',
 u'aberrations',
 u'abi',
 u'abietate',
 u'abilities',
 u'ability',
 u'ablate',
 u'ablated',
 u'ablation',
 u'able',
 u'ablelux',
 u'ably',
 u'abnormal',
 u'abnormalities',
 u'abnormality',
 u'abnormally',
 u'abolished',
 u'abolishes',
 u'aborate',
 u'a

We get a matrix with 800 rows as patent ids and 34534 words as columns. And the intersection of each row and column is the weight of the particular word using the TFID vectorizer.

In [41]:
# get the set of entire columns as the vocabulary.
vocab = tfidf_vectorizer.get_feature_names()
vocab_list = [] 
sub_list = []
# combining the patent id, word index and the weight
for i, val in enumerate(sents_ids):
    for word, weight in zip(vocab, tfidf_vectors.toarray()[i]):
        if weight > 0: #weight 0 is of no use
            sub_list = [val,list(vocab).index(word),weight]
            vocab_list += [sub_list]


NameError: name 'tfidf_vectorizer' is not defined

Store all the weights in a separate text file.

In [ ]:
tf_idfFile = csv.writer(open("tf_idf_vectors.txt","w"))
for value in vocab_list:
    # reading every value and writing them into a file
    tf_idfFile.writerow(value)


## Count Vectors and Binary Vectors.
To use count vectors, we make use of the CountVectorizer. To find the binary vector, we use count vector weight. If count vector weight is greater than 0, we set the binary vector weight to 1.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(input = 'content',analyzer = "word")

In [ ]:
#fitting the sentences into the vectorizer
data_features = vectorizer.fit_transform(sents_words)
print data_features.shape

In [ ]:
countvocab = vectorizer.get_feature_names()
countvocab_list = [] 
countsub_list = []
binaryvocab_list = []
binarysub_list = []

#combining the patent id, word index and the weight for count and binary vector
for i, val in enumerate(sents_ids):
    for word, weight in zip(countvocab, data_features.toarray()[i]):
        if weight > 0:
            countsub_list = [val,list(countvocab).index(word),weight]
            binarysub_list = [val,list(countvocab).index(word),1]
            countvocab_list += [countsub_list]
            binaryvocab_list += [binarysub_list]

In [ ]:
#create a separate file for count vectors
countVectorsFile = csv.writer(open("count_vectors.txt","w"))
for value in countvocab_list:
    # reading every key value and writing them into a file
    countVectorsFile.writerow(value)

In [ ]:
# create a separate file for binary vectors
with open("binary_vectors.txt", "wb") as outfile:
    writer = csv.writer(outfile, delimiter = ",")
    #for each dictionary item (key and value) their respective value are written in the file
    for val in binaryvocab_list:
        writer.writerow(val)

## Task 4

Running SVM classifier without removing stop words.

In [ ]:
!python svm_classifier.py count_vectors.txt section_labels.txt

AUC 0.66

In [ ]:
!python svm_classifier.py binary_vectors.txt  section_labels.txt

AUC 0.67

In [ ]:
!python svm_classifier.py tf_idf_vectors.txt  section_labels.txt

AUC 0.75

Retokenising the multi word token set by removing the stop words

In [45]:
filtered_sents_retoken = {}
for keys in mwe_tokens.iterkeys():
    filtered_sents_retoken[keys] = [token for token in mwe_tokens[keys] if token not in stopwords_list ]

In [46]:
filtered_sents_retoken['07640605'].find

[u'end',
 u'support',
 u'shower',
 u'head',
 u'coordinated',
 u'wall',
 u'may',
 u'pivoted',
 u'horizontal',
 u'axis',
 u'holder',
 u'shower',
 u'head',
 u'may',
 u'inserted',
 u'provided',
 u'far',
 u'end',
 u'arm',
 u'holder',
 u'may',
 u'also',
 u'pivoted',
 u'horizontal',
 u'axis',
 u'background',
 u'invention',
 u'invention',
 u'based',
 u'arrangement',
 u'shower',
 u'head',
 u'may',
 u'detachable',
 u'desired',
 u'supported',
 u'may',
 u'pivoted',
 u'shower',
 u'head',
 u'may',
 u'attached',
 u'console',
 u'using',
 u'example',
 u'conical',
 u'holders',
 u'whose',
 u'heights',
 u'adjustable',
 u'rod',
 u'generally',
 u'known',
 u'rod',
 u'allows',
 u'adjusting',
 u'height',
 u'shower',
 u'head',
 u'suit',
 u'heights',
 u'user',
 u'various',
 u'heights',
 u'pivoting',
 u'order',
 u'change',
 u'angle',
 u'inclination',
 u'shower',
 u'fixture',
 u'pivotable',
 u'shower',
 u'head',
 u'attached',
 u'far',
 u'end',
 u'arm',
 u'turn',
 u'attached',
 u'console',
 u'pivotable',
 u'thereon

Now that the stop words are removed, we generate a new set of senetence for task 3 above by using the below function.

In [50]:
#list of patent ids
sents_ids = []
#list of sentences
sents_words =[]
#using a different set of tokens after removing stop words
for key, value in filtered_sents_retoken.items():
    sents_ids.append("{0}".format(key))
    txt = ' '.join(value) # joining all the tokens to form a sentence
    sents_words.append(txt) # appending the sentences to the list

By running all the vectors again from task 3, different sets of weights are generated.

In [ ]:
!python svm_classifier.py count_vectors.txt section_labels.txt

AUC 0.74

In [ ]:
!python svm_classifier.py binary_vectors.txt  section_labels.txt

AUC 0.68

In [ ]:
!python svm_classifier.py tf_idf_vectors.txt  section_labels.txt

AUC 0.74

When stop words are removed, AUC value increases.